In [0]:
%run ./variables

## Staging layer

In [0]:
spark.sql(f"""
  CREATE VOLUME IF NOT EXISTS {catalog_name}.{bronze_schema_name}.{staging_volume_name}
""")

In [0]:
dbutils.fs.mkdirs(customers_volume_path,)
dbutils.fs.mkdirs(employees_volume_path)
dbutils.fs.mkdirs(municipalities_volume_path)
dbutils.fs.mkdirs(neighborhoods_volume_path)
dbutils.fs.mkdirs(sales_volume_path)

## Bronze layer

In [0]:
spark.sql(f"""
    CREATE SCHEMA IF NOT EXISTS {catalog_name}.{bronze_schema_name}
""")

In [0]:
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {catalog_name}.{bronze_schema_name}.brz_customers (
        customer_id BIGINT,
        name STRING,
        phone STRING,
        email STRING,
        address STRING
    )
""")

In [0]:
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {catalog_name}.{bronze_schema_name}.brz_employees (
        employee_id BIGINT,
        name STRING,
        phone STRING,
        email STRING,
        address STRING,
        comission DOUBLE
    )
""")

In [0]:
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {catalog_name}.{bronze_schema_name}.brz_municipalities (
        DPTOMPIO STRING,
        DPTO_CCDGO STRING,
        MPIO_CCDGO STRING,
        MPIO_CNMBR STRING,
        MPIO_CCNCT STRING,
        geometry BINARY
    )
""")

In [0]:
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {catalog_name}.{bronze_schema_name}.brz_neighborhoods (
        OBJECTID BIGINT,
        CODIGO STRING,
        NOMBRE STRING,
        IDENTIFICACION STRING,
        LIMITEMUNICIPIOID STRING,
        SUBTIPO_COMUNACORREGIMIENTO BIGINT,
        LINK_DOCUMENTO INT,
        SHAPEAREA DOUBLE,
        SHAPELEN DOUBLE,
        geometry BINARY
    )
""")

In [0]:
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {catalog_name}.{bronze_schema_name}.brz_sales (
        latitude DOUBLE,
        longitude DOUBLE,
        date STRING,
        customer_id INT,
        employee_id INT,
        quantity_products INT,
        order_id STRING
    )
""")

## Silver layer

In [0]:
spark.sql(f"""
    CREATE SCHEMA IF NOT EXISTS {catalog_name}.{silver_schema_name}
""")

In [0]:
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {catalog_name}.{silver_schema_name}.slv_customers (
        customer_id BIGINT,
        name STRING,
        phone STRING,
        email STRING,
        address STRING
    )
""")

In [0]:
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {catalog_name}.{silver_schema_name}.slv_employees (
        employee_id BIGINT,
        name STRING,
        phone STRING,
        email STRING,
        address STRING,
        comission DOUBLE
    )
""")

In [0]:
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {catalog_name}.{silver_schema_name}.slv_municipalities (
        code STRING,
        department_code STRING,
        municipality_code STRING,
        municipality STRING,
        ccnct_code STRING,
        geojson STRING
    )
""")

In [0]:
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {catalog_name}.{silver_schema_name}.slv_neighborhoods (
        object_id BIGINT,
        code STRING,
        name STRING,
        identification STRING,
        limit_municipality_id STRING,
        subtype_community_district BIGINT,
        link_document INT,
        shape_area DOUBLE,
        shape_len DOUBLE,
        geojson STRING
    )
""")

In [0]:
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {catalog_name}.{silver_schema_name}.slv_sales (
        order_id STRING,
        customer_id INT,
        employee_id INT,
        event_date TIMESTAMP,
        latitude DOUBLE,
        longitude DOUBLE,
        quantity_products INT,
        partition_date STRING,
        event_day INT,
        event_hour INT,
        event_minute INT,
        event_month INT,
        event_second INT,
        event_year INT
    )
""")